In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
currency_pair = 'Eur_Usd'
time_frame = 'M5'
year_range = '2022-2023'
rounding = 3 if 'Jpy' in currency_pair else 5
pips_multiplier = 100 if 'Jpy' in currency_pair else 10000
fractal_window = 20
divider = 100 if 'Jpy' in currency_pair else 10000

In [3]:
df = pd.read_csv(f'../bar_movement/data/Oanda_{currency_pair}_{time_frame}_{year_range}.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.reset_index(drop=True, inplace=True)

In [4]:
def squeeze(barsdata, length=20, length_kc=20, mult=2.0):
    # Bollinger bands
    m_avg = barsdata['Mid_Close'].rolling(window=length).mean()
    m_std = barsdata['Mid_Close'].rolling(window=length).std(ddof=0)
    upper_bb = m_avg + mult * m_std
    lower_bb = m_avg - mult * m_std

    # Keltner channel
    tr0 = abs(barsdata['Mid_High'] - barsdata['Mid_Low'])
    tr1 = abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    tr2 = abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    range_ma = tr.rolling(window=length_kc).mean()
    upper_kc = m_avg + range_ma * mult
    lower_kc = m_avg - range_ma * mult

    # Squeeze
    squeeze_on = (lower_bb > lower_kc) & (upper_bb < upper_kc)

    return squeeze_on

def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).mean()

def atr_bands(high, low, close, lookback=14, atr_multiplier=3):
    scaled_atr_vals = atr(high, low, close, lookback) * atr_multiplier
    lower_band = close - scaled_atr_vals
    upper_band = close + scaled_atr_vals

    return lower_band, upper_band

def fractal(lows, highs, window=20):
    assert len(lows) == len(highs)

    fractal_period = 2 * window + 1

    is_support = lows.rolling(fractal_period, center=True).apply(lambda x: x[window] == min(x), raw=True)
    is_resistance = highs.rolling(fractal_period, center=True).apply(lambda x: x[window] == max(x), raw=True)
    
    is_support_indices = pd.Series(is_support.index[is_support == 1.0])
    is_resistance_indices = pd.Series(is_resistance.index[is_resistance == 1.0])

    support_fractal_vals = lows[is_support_indices].reindex(lows.index).ffill()
    resistance_fractal_vals = highs[is_resistance_indices].reindex(highs.index).ffill()

    return support_fractal_vals, resistance_fractal_vals

def choc(closes, support_fractals, resistance_fractals):
    broke_down = closes < support_fractals
    broke_up = closes > resistance_fractals

    assert len(broke_down) == len(broke_up)
    
    choc, prev_val, levels = [], None, []

    for i in range(len(broke_down)):
        # First occurrence
        if prev_val is None:
            if broke_down[i]:
                prev_val = 'broke_down'
                choc.append('broke_down')
                levels.append(support_fractals[i])

            elif broke_up[i]:
                prev_val = 'broke_up'
                choc.append('broke_up')
                levels.append(resistance_fractals[i])

            else:
                choc.append('na')
                levels.append(None)

        # All other occurences
        elif broke_down[i] and prev_val == 'broke_up':
            prev_val = 'broke_down'
            choc.append('broke_down')
            levels.append(support_fractals[i])

        elif broke_up[i] and prev_val == 'broke_down':
            prev_val = 'broke_up'
            choc.append('broke_up')
            levels.append(resistance_fractals[i])

        else:
            choc.append('na')
            levels.append(None)

    return pd.Series(choc), pd.Series(levels)

In [5]:
df['lower_atr_band'], df['upper_atr_band'] = atr_bands(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['squeeze'] = squeeze(df)
df['support_fractal'], df['resistance_fractal'] = fractal(df['Mid_Low'], df['Mid_High'], fractal_window)
df['support_fractal'], df['resistance_fractal'] = df['support_fractal'].shift(fractal_window), df['resistance_fractal'].shift(fractal_window)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df['choc'], df['choc_level'] = choc(df['Mid_Close'], df['support_fractal'], df['resistance_fractal'])

In [11]:
value_per_pip = 1.0
amounts_per_day = [-0.008, -0.01, -0.012] if 'Jpy' in currency_pair else [-0.00008, -0.0001, -0.00012]

In [12]:
def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def run_simulation(max_n_bars_for_double_break, invert):
    reward, day_fees, n_wins, n_losses, win_streak, loss_streak, curr_win_streak, curr_loss_streak, n_buys, n_sells = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    pips_risked, win_amounts, loss_amounts = [], [], []
    trade, prev_choc_val, prev_choc_level, n_bars_since_choc, lower_bands, upper_bands = None, None, None, 0, [], []

    for i in range(2, len(df)):
        curr_date, curr_ao, curr_bo, curr_mid_open, curr_ask_low, curr_bid_high, curr_bid_low, curr_ask_high, curr_bid_close, curr_ask_close = df.loc[df.index[i], ['Date', 'Ask_Open', 'Bid_Open', 'Mid_Open', 'Ask_Low', 'Bid_High', 'Bid_Low', 'Ask_High', 'Bid_Close', 'Ask_Close']]
        spread = abs(curr_ao - curr_bo)
        
        mid_close, choc, choc_level, lower_atr_band, upper_atr_band, squeeze_on = df.loc[df.index[i - 1], ['Mid_Close', 'choc', 'choc_level', 'lower_atr_band', 'upper_atr_band', 'squeeze']]

        # Set any new choc variables, if applicable
        if choc != 'na' and squeeze_on:
            assert choc_level is not None

            prev_choc_val, prev_choc_level, n_bars_since_choc, lower_bands, upper_bands = choc, choc_level, 0, [lower_atr_band], [upper_atr_band]

        if prev_choc_val is not None and n_bars_since_choc > 0 and ((mid_close < prev_choc_level and prev_choc_val == 'broke_up') or (mid_close > prev_choc_level and prev_choc_val == 'broke_down')):
            buy_signal = prev_choc_val == 'broke_up'
                            
            if invert:
                buy_signal = not buy_signal

            # Place trade
            if buy_signal:
                open_price = float(curr_ao)
                stop_loss = round(min(lower_bands), rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * 0.1:
                        n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                        
                        pips_risked.append(curr_pips_to_risk)
                        n_buys += 1
                        prev_choc_val, prev_choc_level, n_bars_since_choc, lower_bands, upper_bands = None, None, 0, [], []

            else:
                open_price = float(curr_bo)
                stop_loss = round(max(upper_bands), rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * 0.1:
                        n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                 'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                 'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                        
                        pips_risked.append(curr_pips_to_risk)
                        n_sells += 1
                        prev_choc_val, prev_choc_level, n_bars_since_choc, lower_bands, upper_bands = None, None, 0, [], []

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_close > trade['open_price']:
            curr_profit_ratio = (curr_bid_close - trade['open_price']) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_close < trade['open_price']:
            curr_profit_ratio = (trade['open_price'] - curr_ask_close) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * trade['prev_profit_ratio'])

        if prev_choc_val is not None:
            n_bars_since_choc += 1
            lower_bands.append(lower_atr_band)
            upper_bands.append(upper_atr_band)

            if n_bars_since_choc >= max_n_bars_for_double_break:
                prev_choc_val, prev_choc_level, n_bars_since_choc, lower_bands, upper_bands = None, None, 0, [], []

    return reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts

In [13]:
max_n_bars_for_double_break_vals = [6, 12, 18, 24]
invert_vals = [True, False]

all_combos = []

for max_n_bars_for_double_break in max_n_bars_for_double_break_vals:
    for invert in invert_vals:
        all_combos.append((max_n_bars_for_double_break, invert))

best_max_n_bars_for_double_break, best_invert_val = None, None
top_n_results, best_rewards, best_reward, runs_finished = 10, [], -np.inf, 0

percentage_to_try = 1.0
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

for max_n_bars_for_double_break, invert in combos_to_try:
    reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts = run_simulation(max_n_bars_for_double_break, invert)
    runs_finished += 1

    print(reward, day_fees, reward + day_fees)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print('Min win amount: ' +  str(min(win_amounts)))
        print('Max win amount: ' + str(max(win_amounts)))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print('Min loss amount: ' +  str(min(loss_amounts)))
        print('Max loss amount: ' + str(max(loss_amounts)))

    print('Remaining runs: ' + str(n_runs - runs_finished))

    total_profit = reward + day_fees

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or total_profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(total_profit),'max_n_bars_for_double_break': max_n_bars_for_double_break, 'invert': invert})

    if total_profit > best_reward:
        best_reward = total_profit
        best_max_n_bars_for_double_break, best_invert_val = max_n_bars_for_double_break, invert
 
    print('Best reward so far: ' + str(best_reward))
    print()


Num runs: 8

-1549.960969999971 -44.44100000000001 -1594.401969999971
Num buys: 56
Num sells: 55
Num trades: 111
Num wins: 17
Num losses: 63
Win streak: 5
Loss streak: 14
Avg pips risked: 0.002037117117117105
Avg win amount: 94.1164447058823
Min win amount: 49.99749999999894
Max win amount: 249.99945000000034
Avg loss amount: -36.62721546511591
Min loss amount: -49.999979999996235
Max loss amount: 0.0
Remaining runs: 7
Best reward so far: -1594.401969999971

-1649.9630999999943 -38.814820000000005 -1688.7779199999943
Num buys: 66
Num sells: 62
Num trades: 128
Num wins: 14
Num losses: 61
Win streak: 3
Loss streak: 22
Avg pips risked: 0.0028333593749999986
Avg win amount: 99.99682000000041
Min win amount: 49.99695999999909
Max win amount: 349.99271999999985
Avg loss amount: -40.13050763157895
Min loss amount: -49.99999999999796
Max loss amount: 0.0
Remaining runs: 6
Best reward so far: -1594.401969999971

-1149.9616499999852 -25.771020000000004 -1175.732669999985
Num buys: 45
Num sells: 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best pips to risk: ' + str(best_max_n_bars_for_double_break))
print('Best adapt errors val: ' + str(best_invert_val))